In [2]:
from cvxopt import matrix, solvers
import numpy as np

## Soft-Margin SVM

$$
min_\alpha \frac{1}{2}\sum_{m=1}^N\sum_{n=1}^N\alpha_n\alpha_my_ny_mx_n^Tx_m - \sum_{n=1}^N\alpha_n
$$

$$
s.t.\ \sum_{n=1}^Ny_n\alpha_n = 0
$$

$$
\ \ \ \ \ \ 0 \leq \alpha_n \leq C
$$

$$
n=1,\dots,N
$$

## QP Solver Representation

```python3
solvers.qp(Q, p, G, h, A, b)
```

Where the problem is:
$$
min_x \frac{1}{2}x^TQx + p^Tx
$$

$$
s.t.\ Gx \leq h
$$

$$
\ \ \ \ \ \ Ax = b
$$

## What to do

We need to convert Soft-Margin SVM representation into QP solver one.

In [ ]:
class Dataset:
    """
    This class is taken directly from our previous submission.
    """
    def __init__(self, train_path=None, val_path=None, test_path=None):
        self.train_path = train_path
        self.val_path = val_path
        self.test_path = test_path
    
    def load_csv(self, path):
        data = pd.read_csv(path).to_numpy()
        X, Y_str = data[:, :-1], data[:, -1]  # separate data and target
        n_examples = len(Y_str)
        Y = np.zeros(n_examples)
        for i in range(n_examples):
            category = Y_str[i]
            if category == "banana":
                Y[i] = 0
            elif category == "carrot":
                Y[i] = 1
            elif category == "cucumber":
                Y[i] = 2
            elif category == "mandarin":
                Y[i] = 3
            else:
                Y[i] = 4
        
        return X.astype(float), Y.astype(float)
    
    def get_data(self):
        X_train, Y_train = self.load_csv(self.train_path)
        X_val, Y_val     = self.load_csv(self.val_path)
        X_test, Y_test   = self.load_csv(self.test_path)
        
        return (X_train, Y_train), (X_val, Y_val), (X_test, Y_test)

In [ ]:
class SoftMarginSVM:
    def __init__(self, C):
        self.weights = None
        self.bias    = None
        self.C       = C  # soft margin svm gets closer to hard when C gets greater
        
    def train(self, X, Y):
        n_examples, n_features = X.shape

        # we need to calculate Q using y_n*y_m and x_n^Tx_m
        Y = Y.reshape(-1, 1)    # to prevent np to give a scalar
        Y_mul = np.dot(Y, Y.T)  # NxN
        X_mul = np.dot(X, X.T)  # NxN

        Q = Y_mul * X_mul
        p = (np.ones(n_examples) * -1).reshape(-1, 1)

        # G is supposed to be (2N)xN
        G_first_half  = np.eye(n_examples) * -1  # greater than or equal to 0
        G_second_half = np.eye(n_examples)       # less than or equal to C
        G = np.vstack([G_first_half, G_second_half])
        h_first_half  = np.zeros(n_examples).reshape(-1, 1)
        h_second_half = (np.ones(n_examples) * self.C).reshape(-1, 1)
        h = np.vstack([h_first_half, h_second_half])

        A = Y.reshape(1, -1)
        b = 0.0  # cvxopt expects double-precision

        sol=solvers.qp(matrix(Q), matrix(p), matrix(G), matrix(h), matrix(A), matrix(b))

        # next: use sol["x"] to extract optimal alphas and then calculate w and b